In [1]:
import nibabel as nib
import numpy as np
import os
from os import path
import pandas as pd
from scipy.spatial import distance_matrix
from sklearn.cluster import DBSCAN
import sys
from zipfile import ZipFile

In [2]:
from fsedit_notes_helpers import get_diff_data, summarize_edits, get_bm_edits, get_wm_edits, get_cp_edits, get_bfs_edits

In [3]:
subnum = 'CC0117_core1'
typ = 'bm'
basedir='/Users/zeynepenkavi/Downloads/ConteQC'
outdir='/Users/zeynepenkavi/Downloads/ConteQC'

In [4]:
editp = path.join(basedir, 'sub-'+subnum)
uneditp = path.join(basedir, 'sub-'+subnum+'_unedited')
typ_dict = {'bm': 'brainmask', 'wm': 'wm', 'bfs': 'brain.finalsurfs', 'cp': 'control.dat'}
vol = typ_dict[typ]

In [5]:
exists = path.isfile(path.join(basedir, 'sub-'+subnum+'.zip'))
if not exists:
    print("Unedited images not found in directory. Quitting...")
    try:
        sys.exit()
    except SystemExit:
        print("Quit because difference images cannot be calculated without unedited images.")
else:
    # Unzip unedited images of potential interest
    toExtract = ['brainmask.mgz', 'wm.mgz', 'brain.finalsurfs.mgz']
    suffix = path.join('sub-'+subnum, 'mri')
    toExtract = [path.join(suffix, s) for s in toExtract]
    with ZipFile(path.join(basedir, 'sub-'+subnum+'.zip'), 'r') as zipObj:
        # Get a list of all archived file names from the zip
        listOfFileNames = zipObj.namelist()
        # Iterate over the file names
        for fileName in listOfFileNames:
            # Check filename endswith csv
            if fileName in toExtract:
                # Extract a single file from zip
                zipObj.extract(fileName, path.join(basedir, 'sub-'+subnum+'_unedited'))

In [6]:
typ = "wm"
vol = typ_dict[typ]
get_wm_edits(editp, uneditp, vol, subnum)

,cluster,min_sag,max_sag,min_axe,max_axe,min_cor,max_cor,num_vox,vol,action
0,-1,110,112,134,136,137,137,4,wm,add voxel
0,-1,111,112,135,136,137,137,4,wm,delete voxel


In [ ]:
tmp = get_diff_data(editp, uneditp, vol, subnum)
cur_diff_data = tmp.query("Action=='add voxel'")

In [ ]:
cur_diff_data

In [ ]:
dm = distance_matrix(cur_diff_data.loc[:,('Sag', 'Axe', 'Cor')], cur_diff_data.loc[:,('Sag', 'Axe', 'Cor')])

In [ ]:
clustering = DBSCAN(eps=50, min_samples=5).fit(dm)

In [ ]:
list(clustering.labels_)

In [ ]:
#cur_diff_data['cluster'] = list(clustering.labels_)
cur_diff_data.insert(0, "cluster", list(clustering.labels_), True)

In [ ]:
cur_diff_data

In [ ]:
summarize_edits(tmp)

In [ ]:
get_bm_edits(editp, uneditp, vol, subnum)

In [ ]:
typ = "cp"
vol = typ_dict[typ]
get_cp_edits(editp, uneditp, vol, subnum)

In [ ]:
typ = "bfs"
vol = typ_dict[typ]
get_bfs_edits(editp, uneditp, vol, subnum)

In [ ]:
fn_dict = {'bm': get_bm_edits, 'wm': get_wm_edits, 'cp': get_cp_edits, 'bfs': get_bfs_edits}
typ = "all"
if typ == 'all':
    out = pd.DataFrame()
    # loop through function lookup dictionary and run all edit extraction functions
    for k,v in fn_dict.items():
        vol = typ_dict[k]
        out = out.append(v(editp=editp, uneditp=uneditp, vol=vol, subnum=subnum))

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
out

To do still: 

how will this be used? 
no notes taken while editing
run this for subject after editing to get notes
[present output in slack channel before switching to this fully]

next step i need to do: 
refine previous edits

can this be useful for that?
could at least run all subjects through this and see if i've missed writing down any edits i've made

how should i go about refining edits?
go over mike's reports and compare for each subject how different i am from dorit/mike
run their edited images (taken from box) through 

so timeline:
finish this script
    -go over current output and see if it misses anything for bm
    -write other volume edits extractions
run all subjects through this and see if you've missed recording any edits
move onto refinement of previous edits:
    -download dorit's edits from box and make difference images for 
    -overlay original, my edits difference image, dorit's difference image in freeview and scroll through
move onto editing new subjects